In [1]:
import pandas as pd
import glob

In [2]:
path = r'Desktop/sumoDemo'
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
sumoR = pd.concat(dfs, ignore_index=True)

In [3]:
new = sumoR[['rank']].copy()
# <p><b>1</b>: A 1 or 2 letter code for the rikishi's division or makuuchi rank:\n</p>,
# <p><b>2</b>: Number within division or rank, or H for sanyaku and  maegashira haridashi<br/>\n<b>3</b>: <b>E</b> or <b>W</b> for East or West\n</p>]
new['rank'] = new['rank'].str.replace('\d+', '')
new['rank'] = new['rank'].str.replace('e', '')
new['rank'] = new['rank'].str.replace('w', '')
new=new.drop_duplicates()
new=new.rename(index=str, columns={"rank": "Rank_Code"})
new

,Rank_Code
0,Y
2,YHD
3,O
5,OHD
6,S
8,SHD
9,K
11,M
38,J
64,Ms


In [4]:
import requests
from bs4 import BeautifulSoup as bs

In [5]:
url="https://en.m.wikipedia.org/wiki/Template:Sumo_rank"
response = requests.get(url)
soup=bs(response.text,'html.parser')
thangs=soup.select('p')
thangs

[<p>This constructs a string for a sumotori's rank. It takes three unnamed parameters. All are optional, but if none are given the results are very uninteresting. It's primarily intended for use by other templates, but it may have a broader application.\n</p>,
 <p><b>1</b>: A 1 or 2 letter code for the rikishi's division or makuuchi rank:\n</p>,
 <p><b>2</b>: Number within division or rank, or H for sanyaku and  maegashira haridashi<br/>\n<b>3</b>: <b>E</b> or <b>W</b> for East or West\n</p>]

In [6]:
url="https://en.m.wikipedia.org/wiki/Template:Sumo_rank"
response = requests.get(url)
soup=bs(response.text,'html.parser')
things=soup.select('li')
things

[<li class="mw-ui-icon mw-ui-icon-element mw-ui-icon-minerva-edit" id="ca-edit" title="Edit the lead section of this page"><a class="edit-page" href="/w/index.php?title=Template:Sumo_rank&amp;action=edit&amp;section=0">Edit</a></li>,
 <li class="mw-ui-icon mw-ui-icon-element mw-ui-icon-minerva-watch watch-this-article" id="ca-watch"><a accesskey="w" href="/w/index.php?title=Special:UserLogin&amp;returnto=Template%3ASumo+rank" title="Add this page to your watchlist [w]"></a></li>,
 <li class="mw-ui-icon mw-ui-icon-element mw-ui-icon-minerva-language-switcher language-selector" title="Read in another language"><a href="/wiki/Special:MobileLanguages/Template:Sumo_rank">Read in another language</a></li>,
 <li><b>Y</b>: Yokozuna</li>,
 <li><b>O</b>: \u014czeki</li>,
 <li><b>S</b>: Sekiwake</li>,
 <li><b>K</b>: Komosubi</li>,
 <li><b>M</b>: Maegashira</li>,
 <li><b>J</b>: J\u016bry\u014d</li>,
 <li><b>MA</b>: Makushita</li>,
 <li><b>MT</b>: Makushita tsukedashi</li>,
 <li><b>SA</b>: Sandanme

In [7]:
len(things)

21

In [8]:
rankdf=pd.DataFrame()
x=0
for row in things[1:]:
        rankdf.loc[x, 'Yokozuna'] = row.text
        x += 1

In [9]:
rankdf=rankdf.drop([0,1,17,18,19])

In [10]:
rankdf['rank'] = rankdf['Yokozuna'].apply(lambda s:s.split(':')[1])

In [11]:
rankdf['code'] = rankdf['Yokozuna'].apply(lambda s:s.split(':')[0])

In [12]:
rankdf.drop(columns=['Yokozuna'])

,rank,code
2,Yokozuna,Y
3,Ōzeki,O
4,Sekiwake,S
5,Komosubi,K
6,Maegashira,M
7,Jūryō,J
8,Makushita,MA
9,Makushita tsukedashi,MT
10,Sandanme,SA
11,Sandanme tsukedashi,ST


In [13]:
finaldf = pd.merge(rankdf, new,
                           left_on='code', right_on='Rank_Code',
                           how='outer'
                           ) 

In [14]:
finaldf

,Yokozuna,rank,code,Rank_Code
0,Y: Yokozuna,Yokozuna,Y,Y
1,O: Ōzeki,Ōzeki,O,O
2,S: Sekiwake,Sekiwake,S,S
3,K: Komosubi,Komosubi,K,K
4,M: Maegashira,Maegashira,M,M
5,J: Jūryō,Jūryō,J,J
6,MA: Makushita,Makushita,MA,NaN
7,MT: Makushita tsukedashi,Makushita tsukedashi,MT,NaN
8,SA: Sandanme,Sandanme,SA,NaN
9,ST: Sandanme tsukedashi,Sandanme tsukedashi,ST,NaN


In [19]:
finaldf=finaldf.dropna()

In [21]:
finaldf=finaldf[['rank', 'Rank_Code']].copy()
finaldf=finaldf.rename(index=str, columns={"rank": "Rank"})

finaldf

,Rank,Rank_Code
0,Yokozuna,Y
1,Ōzeki,O
2,Sekiwake,S
3,Komosubi,K
4,Maegashira,M
5,Jūryō,J
